In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('online').getOrCreate()

In [3]:
df=spark.read.csv('online_shoppers_intention.csv',inferSchema=True,header=True)
df.head(5)

[Row(Administrative=0, Administrative_Duration=0.0, Informational=0, Informational_Duration=0.0, ProductRelated=1, ProductRelated_Duration=0.0, BounceRates=0.2, ExitRates=0.2, PageValues=0.0, SpecialDay=0.0, Month='Feb', OperatingSystems=1, Browser=1, Region=1, TrafficType=1, VisitorType='Returning_Visitor', Weekend=False, Revenue=False),
 Row(Administrative=0, Administrative_Duration=0.0, Informational=0, Informational_Duration=0.0, ProductRelated=2, ProductRelated_Duration=64.0, BounceRates=0.0, ExitRates=0.1, PageValues=0.0, SpecialDay=0.0, Month='Feb', OperatingSystems=2, Browser=2, Region=1, TrafficType=2, VisitorType='Returning_Visitor', Weekend=False, Revenue=False),
 Row(Administrative=0, Administrative_Duration=0.0, Informational=0, Informational_Duration=0.0, ProductRelated=1, ProductRelated_Duration=0.0, BounceRates=0.2, ExitRates=0.2, PageValues=0.0, SpecialDay=0.0, Month='Feb', OperatingSystems=4, Browser=1, Region=9, TrafficType=3, VisitorType='Returning_Visitor', Weekend

In [4]:
df.columns

['Administrative',
 'Administrative_Duration',
 'Informational',
 'Informational_Duration',
 'ProductRelated',
 'ProductRelated_Duration',
 'BounceRates',
 'ExitRates',
 'PageValues',
 'SpecialDay',
 'Month',
 'OperatingSystems',
 'Browser',
 'Region',
 'TrafficType',
 'VisitorType',
 'Weekend',
 'Revenue']

In [5]:
display(df)

DataFrame[Administrative: int, Administrative_Duration: double, Informational: int, Informational_Duration: double, ProductRelated: int, ProductRelated_Duration: double, BounceRates: double, ExitRates: double, PageValues: double, SpecialDay: double, Month: string, OperatingSystems: int, Browser: int, Region: int, TrafficType: int, VisitorType: string, Weekend: boolean, Revenue: boolean]

In [6]:
df.printSchema()

root
 |-- Administrative: integer (nullable = true)
 |-- Administrative_Duration: double (nullable = true)
 |-- Informational: integer (nullable = true)
 |-- Informational_Duration: double (nullable = true)
 |-- ProductRelated: integer (nullable = true)
 |-- ProductRelated_Duration: double (nullable = true)
 |-- BounceRates: double (nullable = true)
 |-- ExitRates: double (nullable = true)
 |-- PageValues: double (nullable = true)
 |-- SpecialDay: double (nullable = true)
 |-- Month: string (nullable = true)
 |-- OperatingSystems: integer (nullable = true)
 |-- Browser: integer (nullable = true)
 |-- Region: integer (nullable = true)
 |-- TrafficType: integer (nullable = true)
 |-- VisitorType: string (nullable = true)
 |-- Weekend: boolean (nullable = true)
 |-- Revenue: boolean (nullable = true)



In [8]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [66]:
feat_cols=['Administrative','Administrative_Duration','Informational','Informational_Duration','ProductRelated','ProductRelated_Duration','BounceRates','ExitRates','PageValues','SpecialDay','OperatingSystems','Browser','Region','TrafficType']

In [10]:
vec_assembler=VectorAssembler(inputCols=feat_cols,outputCol='features')
final_data=vec_assembler.transform(df)

KNN

In [11]:
from pyspark.ml.feature import StandardScaler

In [12]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [13]:
evaluator=ClusteringEvaluator

In [14]:
kmeans=KMeans().setK(3).setSeed(1)
model=kmeans.fit(final_data)

In [15]:
predictions=model.transform(final_data)

In [17]:
final_data.head()

Row(Administrative=0, Administrative_Duration=0.0, Informational=0, Informational_Duration=0.0, ProductRelated=1, ProductRelated_Duration=0.0, BounceRates=0.2, ExitRates=0.2, PageValues=0.0, SpecialDay=0.0, Month='Feb', OperatingSystems=1, Browser=1, Region=1, TrafficType=1, VisitorType='Returning_Visitor', Weekend=False, Revenue=False, features=SparseVector(15, {4: 1.0, 6: 0.2, 7: 0.2, 10: 1.0, 11: 1.0, 12: 1.0, 13: 1.0}))

In [18]:
centers=model.clusterCenters()

In [19]:
for center in centers:
    print(center)

[1.82098219e+00 6.20382794e+01 3.28018896e-01 1.99896921e+01
 1.86472788e+01 5.78398522e+02 2.54290118e-02 4.76798214e-02
 5.51449251e+00 6.40881803e-02 2.12114949e+00 2.36708985e+00
 3.16937309e+00 4.14772168e+00 2.35114654e-01]
[7.43902439e+00 2.95641709e+02 2.77073171e+00 2.69296003e+02
 2.36985366e+02 1.09227713e+04 5.91820387e-03 1.97062157e-02
 4.54367488e+00 3.12195122e-02 2.14634146e+00 2.31219512e+00
 2.57073171e+00 3.63902439e+00 2.43902439e-01]
[4.33706721e+00 1.55558025e+02 1.17515275e+00 8.48898799e+01
 7.80000000e+01 3.36809846e+03 7.13966435e-03 2.16767600e-02
 7.96860373e+00 5.08146640e-02 2.13645621e+00 2.31008147e+00
 3.09368635e+00 3.71028513e+00 2.18431772e-01]


Decisiong Tree

In [33]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import pandas
from pyspark.ml.feature import OneHotEncoder

In [62]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType

In [63]:
df.withColumn("Revenue",df.Revenue.cast(IntegerType()))

DataFrame[Administrative: int, Administrative_Duration: double, Informational: int, Informational_Duration: double, ProductRelated: int, ProductRelated_Duration: double, BounceRates: double, ExitRates: double, PageValues: double, SpecialDay: double, Month: string, OperatingSystems: int, Browser: int, Region: int, TrafficType: int, VisitorType: string, Weekend: boolean, Revenue: int]

In [103]:
label=StringIndexer(inputCol='Revenue',outputCol='indexLabel').fit(df)

In [104]:
feature=VectorAssembler(inputCols=feat_cols,outputCol='feature')

In [105]:
(trainData,testData)=df.randomSplit([0.7,0.3])

In [106]:
dt=DecisionTreeClassifier(labelCol='indexLabel',featuresCol='feature')

In [107]:
pipeline=Pipeline(stages=[label,feature,dt])

In [108]:
model=pipeline.fit(trainData)

In [109]:
predictions=model.transform(testData)

In [110]:
predictions.select('prediction','indexLabel','feature').show()

+----------+----------+--------------------+
|prediction|indexLabel|             feature|
+----------+----------+--------------------+
|       0.0|       0.0|(14,[6,7,10,11,12...|
|       0.0|       0.0|(14,[6,7,10,11,12...|
|       0.0|       0.0|(14,[4,7,10,11,12...|
|       0.0|       0.0|(14,[4,7,10,11,12...|
|       0.0|       0.0|(14,[4,7,10,11,12...|
|       0.0|       0.0|(14,[4,6,7,10,11,...|
|       0.0|       0.0|(14,[4,6,7,10,11,...|
|       0.0|       0.0|(14,[4,6,7,10,11,...|
|       0.0|       0.0|(14,[4,6,7,10,11,...|
|       0.0|       0.0|(14,[4,6,7,10,11,...|
|       0.0|       0.0|(14,[4,6,7,10,11,...|
|       0.0|       0.0|(14,[4,6,7,10,11,...|
|       0.0|       0.0|(14,[4,6,7,10,11,...|
|       0.0|       0.0|(14,[4,6,7,10,11,...|
|       0.0|       0.0|(14,[4,6,7,10,11,...|
|       0.0|       0.0|(14,[4,6,7,10,11,...|
|       0.0|       0.0|(14,[4,6,7,10,11,...|
|       0.0|       0.0|(14,[4,6,7,10,11,...|
|       0.0|       0.0|(14,[4,6,7,10,11,...|
|       0.